# Praca domowa nr 2
Eksplorowany przez nas zbiór danych dotyczy historii składanych zamówień w serwisie Allegro.

In [154]:
import pandas as pd
import numpy as np
import copy
import category_encoders as ce

from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from verstack.stratified_continuous_split import scsplit
from category_encoders import TargetEncoder
from category_encoders import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

## Wczytanie i sprawdzenie danych

In [155]:
allegro = pd.read_csv('allegro-api-transactions.csv')
allegro.head()

,lp,date,item_id,categories,pay_option_on_delivery,pay_option_transfer,seller,price,it_is_allegro_standard,it_quantity,it_is_brand_zone,it_seller_rating,it_location,main_category
0,0,2016-04-03 21:21:08,4753602474,"['Komputery', 'Dyski i napędy', 'Nośniki', 'No...",1,1,radzioch666,59.99,1,997,0,50177,Warszawa,Komputery
1,1,2016-04-03 15:35:26,4773181874,"['Odzież, Obuwie, Dodatki', 'Bielizna damska',...",1,1,InwestycjeNET,4.90,1,9288,0,12428,Warszawa,"Odzież, Obuwie, Dodatki"
2,2,2016-04-03 14:14:31,4781627074,"['Dom i Ogród', 'Budownictwo i Akcesoria', 'Śc...",1,1,otostyl_com,109.90,1,895,0,7389,Leszno,Dom i Ogród
3,3,2016-04-03 19:55:44,4783971474,"['Książki i Komiksy', 'Poradniki i albumy', 'Z...",1,1,Matfel1,18.50,0,971,0,15006,Wola Krzysztoporska,Książki i Komiksy
4,4,2016-04-03 18:05:54,4787908274,"['Odzież, Obuwie, Dodatki', 'Ślub i wesele', '...",1,1,PPHU_RICO,19.90,1,950,0,32975,BIAŁYSTOK,"Odzież, Obuwie, Dodatki"


## Podział próbki

In [156]:
# Podział warstwowy według price
allegro_train, allegro_test = scsplit(allegro, test_size=0.3, random_state=42, stratify=allegro['price'])
allegro_train = allegro_train.reset_index(drop=True) # resetujemy indeksy wierszy
allegro_test= allegro_test.reset_index(drop=True)

allegro_train_tr,allegro_train_tst =scsplit(allegro_train, test_size=0.3, random_state=42, stratify=allegro['price'])
allegro_train_tr = allegro_train_tr.reset_index(drop=True) # resetujemy indeksy wierszy
allegro_train_tst =allegro_train_tst.reset_index(drop=True)

In [157]:
allegro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420020 entries, 0 to 420019
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   lp                      420020 non-null  int64  
 1   date                    420020 non-null  object 
 2   item_id                 420020 non-null  int64  
 3   categories              420020 non-null  object 
 4   pay_option_on_delivery  420020 non-null  int64  
 5   pay_option_transfer     420020 non-null  int64  
 6   seller                  420020 non-null  object 
 7   price                   420020 non-null  float64
 8   it_is_allegro_standard  420020 non-null  int64  
 9   it_quantity             420020 non-null  int64  
 10  it_is_brand_zone        420020 non-null  int64  
 11  it_seller_rating        420020 non-null  int64  
 12  it_location             420020 non-null  object 
 13  main_category           420020 non-null  object 
dtypes: float64(1), int64

In [158]:
allegro.isnull().sum()

lp                        0
date                      0
item_id                   0
categories                0
pay_option_on_delivery    0
pay_option_transfer       0
seller                    0
price                     0
it_is_allegro_standard    0
it_quantity               0
it_is_brand_zone          0
it_seller_rating          0
it_location               0
main_category             0
dtype: int64

#  Budowanie

## Braki danych

Na pierwszy rzut oka wydaję się, że ramka nie zawiera braków. Trzeba wziąć jednak pod uwagę, że brak danych nie zawsze jest oznaczanu przez *NULL* czy *NaN*

Przyjrzyjmy się zatem kolumnom z wartościami liczbowymi.

In [159]:
allegro_train_tr.describe()

,lp,item_id,pay_option_on_delivery,pay_option_transfer,price,it_is_allegro_standard,it_quantity,it_is_brand_zone,it_seller_rating
count,205809.000000,2.058090e+05,205809.000000,205809.000000,205809.000000,205809.000000,205809.000000,205809.000000,205809.000000
mean,210119.397874,5.808823e+09,0.878436,0.812554,76.688166,0.583731,6773.397014,0.016559,20389.004077
std,121283.204238,3.384299e+08,0.326783,0.390270,429.431272,0.492941,23437.468604,0.127612,36601.764862
min,0.000000,4.415896e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000
25%,105080.000000,5.651919e+09,1.000000,1.000000,8.990000,0.000000,6.000000,0.000000,1570.000000
50%,210146.000000,5.967878e+09,1.000000,1.000000,24.990000,1.000000,95.000000,0.000000,6697.000000
75%,315165.000000,6.070755e+09,1.000000,1.000000,67.990000,1.000000,931.000000,0.000000,21090.000000
max,420019.000000,6.099916e+09,1.000000,1.000000,119000.000000,1.000000,99999.000000,1.000000,292074.000000


Na tej podstawie stwierdzamy, że wartości "podejrzane" to 0 dla kolumny *price*, 0 dla *it_quantity* oraz -1 dla *it_seller_rating*. Zastosujemy imputację medianą.

In [160]:
def impute_missing(df):
    for column in ['price', 'it_quantity', 'it_seller_rating']:
        if column!='it_seller_rating':
            df[column].loc[df[column]==0]=df[column].loc[df[column]!=0].median()
        else:
            df[column].loc[df[column]==-1]=df[column].loc[df[column]!=-1].median()

In [161]:
impute_missing(allegro_train_tr)

## Encoding zmiennej *it-location*
zacznijmy od encodingu zmiennej *it_location*. Najpierw zmienię pierwsze litery rekordów na małe i to samo wykonam dla *main_category*. DO wykonania encodingu użyję TargetEncoder.

In [162]:
allegro_train_tr['it_location']=allegro_train_tr['it_location'].str.lower()
allegro_train_tr['main_category']=allegro_train_tr['main_category'].str.lower()

In [163]:
encoder=TargetEncoder()
allegro_train_tr['it_location']=encoder.fit_transform(allegro_train_tr['it_location'], allegro_train_tr['price'])
allegro_train_tr

,lp,date,item_id,categories,pay_option_on_delivery,pay_option_transfer,seller,price,it_is_allegro_standard,it_quantity,it_is_brand_zone,it_seller_rating,it_location,main_category
0,245512,2016-04-03 16:04:48,6091183856,"['Gry', 'Gry na konsole', 'Pegasus']",1,1,Sajgon6,14.99,0,237,0,514,66.919614,gry
1,112370,2016-04-03 19:33:34,6087575183,"['Odzież, Obuwie, Dodatki', 'Odzież i bielizna...",1,1,TheBestLook,59.99,1,237,0,936,68.514680,"odzież, obuwie, dodatki"
2,277719,2016-04-03 17:23:26,5880072408,"['Telefony i Akcesoria', 'Akcesoria GSM', 'Fol...",1,0,pawelec01,5.40,1,98645,0,5046,90.270269,telefony i akcesoria
3,243899,2016-04-03 09:56:18,5252897056,"['Dom i Ogród', 'Ogród', 'Narzędzia ogrodnicze...",1,1,cencik-tools,28.00,1,3,0,3029,122.458265,dom i ogród
4,111123,2016-04-03 16:13:14,5804280383,"['Telefony i Akcesoria', 'Akcesoria GSM', 'Pok...",1,1,KoshibaZ-T,25.49,1,807,0,4338,91.314765,telefony i akcesoria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205804,113631,2016-04-03 19:10:06,5935546262,"['Motoryzacja', 'Części samochodowe', 'Oświetl...",1,0,AgnaBoas,39.99,1,75,0,225,45.137251,motoryzacja
205805,268456,2016-04-03 14:32:14,6096748337,"['Uroda', 'Kosmetyki pielęgnacyjne', 'Ciało', ...",1,0,ECO-FUD,9.70,1,3,0,270,44.722719,uroda
205806,416308,2016-04-03 15:19:32,5271125404,"['Odzież, Obuwie, Dodatki', 'Odzież damska', '...",1,0,studio_anet,21.98,1,9685,0,29413,72.079900,"odzież, obuwie, dodatki"
205807,253825,2016-04-03 11:48:41,6089547624,"['Motoryzacja', 'Części do maszyn', 'Części do...",1,0,Farmexpert,59.95,0,8,0,1218,15.247662,motoryzacja


## Kategoryzacja zmiennych ciągłych
Zmienne ciągłe w analizowanej ramce danych to *price*, *it_quantity* i *it_seller_rating*. Do budowy naszego modelu wystarczy rozpatrywać ich wartości przedziałami. W przypadku *price*, *it_quantity* odpowiedniną kategorię przyporządkujemy biorąc pod uwagę *main_category* danego rekordu. Przyjrzyjmy się raz jeszcze opisom tych kolumn dla *main_category*. Natomiast w przypadku *it_seller_rating* nie wiemy w jaki sposób są przyznawane punkty za oceny zatem przyjmujemy, że dobry sprzedawca ma rating>=5000, a w przeciwnym przypadku jest zły - punkt podziału szacowałyśmy w okolicach mediany (6711).

In [164]:
allegro_train_tr.groupby(by='main_category').describe().loc[:,["price","it_quantity"]]

price                                 \
                                       count        mean          std   min   
main_category                                                                 
antyki i sztuka                        588.0  111.940357   282.747864  1.00   
bilety                                  61.0   93.526393   184.956019  2.49   
biuro i reklama                       1594.0   33.126399    83.033298  1.00   
biżuteria i zegarki                   2852.0   57.278377   192.764351  1.00   
delikatesy                            3898.0   20.043197    27.625101  1.00   
dla dzieci                           20494.0   70.838040   154.679828  1.00   
dom i ogród                          44603.0   72.170357   186.007540  1.00   
filmy                                  483.0   33.808696    47.177958  1.89   
fotografia                            1171.0  154.009214   523.450129  1.00   
gry                                   3467.0   41.067055    55.380389  1.00   
instrumenty                            282.0  115.828652   252.250745  1.00   
kolekcje                              3023.0   42.723083   124.769069  1.00   
komputery                             7113.0  126.064884   397.353966  1.00   
konsole i automaty                     502.0  128.614462   280.819850  1.00   
książki i komiksy                     5672.0   24.779933    29.045634  1.00   
motoryzacja                          22601.0  133.974500  1179.450192  1.00   
muzyka                                 948.0   35.845549    31.881607  1.00   
nieruchomości                            2.0    1.000000     0.000000  1.00   
odzież, obuwie, dodatki              26612.0   75.560519    79.932272  1.00   
przemysł                              2974.0   76.444929   255.324971  1.00   
rtv i agd                            10035.0  106.556621   283.759071  1.00   
rękodzieło                            3248.0    6.806004    10.514041  1.00   
sport i turystyka                    13581.0   94.068757   210.563632  1.00   
sprzęt estradowy, studyjny i dj-ski    188.0  155.075745   403.600667  1.00   
telefony i akcesoria                  9628.0   80.645158   250.393462  1.00   
uroda                                13810.0   28.330546    43.476411  1.00   
zdrowie                               6379.0   43.629763   123.822496  1.00   

                                                                          \
                                         25%     50%       75%       max   
main_category                                                              
antyki i sztuka                      11.3750  30.000   92.2500    2790.0   
bilety                                3.0000  15.000   90.0000     900.0   
biuro i reklama                       3.8925  12.000   26.9650    1356.0   
biżuteria i zegarki                   5.9000  14.900   44.0000    7000.0   
delikatesy                            5.3900  11.990   24.0000     384.0   
dla dzieci                           12.9900  30.000   79.0000   12500.0   
dom i ogród                           6.0000  18.000   64.9000    7400.0   
filmy                                 9.9900  19.980   37.3450     485.0   
fotografia                           14.9000  29.000   69.9000    9000.0   
gry                                   7.5000  24.900   54.9900    1200.0   
instrumenty                          16.0000  35.000  109.7500    2710.0   
kolekcje                              4.9000  11.500   35.0000    3000.0   
komputery                            11.5000  33.440   79.9900    8948.0   
konsole i automaty                    9.9225  39.000  109.9750    1849.0   
książki i komiksy                     9.5600  19.000   29.1300     545.0   
motoryzacja                          14.9900  37.000   99.0000  119000.0   
muzyka                               17.0000  29.000   40.0500     390.0   
nieruchomości                         1.0000   1.000    1.0000       1.0   
odzież, obuwie, dodatki              19.9900  43.000  110.0000    1349.0

In [165]:
# Funkcje zmieniające zmienne ciągłe na kategoryczne
def categorize_price(df):
    df2 = copy.deepcopy(df)
    for cat in df['main_category'].unique():
        summary =df2.loc[df2['main_category']==cat,"price"].describe()
        if summary[4]!=summary[6]:
            df.loc[df['main_category']==cat,'price'] = pd.cut(df.loc[df['main_category']==cat,'price'],bins=[0,summary[4],summary[6],119000],labels=['low','average','high'])
        else:
             df.loc[df['main_category']==cat,'price'] = pd.cut(df.loc[df['main_category']==cat,'price'],bins=[0,119000],labels=['average'])
    return

def categorize_quantity(df):
    df2 = copy.deepcopy(df)
    for cat in df['main_category'].unique():
        summary =df2.loc[df2['main_category']==cat,"it_quantity"].describe()
        if summary[4]!=summary[6]:
            df.loc[df['main_category']==cat,'it_quantity'] = pd.cut(df.loc[df['main_category']==cat,'it_quantity'],bins=[0,summary[4],summary[6],99999],labels=['low','average','high'])
        else:
             df.loc[df['main_category']==cat,'it_quantity'] = pd.cut(df.loc[df['main_category']==cat,'it_quantity'],bins=[0,119000],labels=['average'])
    return

def categorize_rating(df):
    df['it_seller_rating'] = pd.cut(df['it_seller_rating'],bins=[0,5000,292074],labels=['bad','good'])
    return

In [166]:
categorize_price(allegro_train_tr)
categorize_quantity(allegro_train_tr)
categorize_rating(allegro_train_tr)

Sprawdźmy czy kategoryzacje się powiodły - zmieniane kolumny powinny mieć odpowiednio 3, 3 i 2 unikatowe wartości.

In [167]:
# Sprawdzenie ilości unikatowych wartośći w każdej kolumnie
allegro_train_tr.nunique()

lp                        205809
date                       34455
item_id                   176834
categories                  8022
pay_option_on_delivery         2
pay_option_transfer            2
seller                     37480
price                          3
it_is_allegro_standard         2
it_quantity                    3
it_is_brand_zone               2
it_seller_rating               2
it_location                 3913
main_category                 27
dtype: int64

Jak widać kategoryzacja się powiodła.

### Encoding zmiennych ciągłych po kategoryacji

In [168]:
def encoding(df):
    le = LabelEncoder()
    for column in ['price', 'it_seller_rating', 'it_quantity']:    
        df[column] = le.fit_transform(df[column])               
    
    return 

In [169]:
encoding(allegro_train_tr)
allegro_train_tr

,lp,date,item_id,categories,pay_option_on_delivery,pay_option_transfer,seller,price,it_is_allegro_standard,it_quantity,it_is_brand_zone,it_seller_rating,it_location,main_category
0,245512,2016-04-03 16:04:48,6091183856,"['Gry', 'Gry na konsole', 'Pegasus']",1,1,Sajgon6,0,0,0,0,0,66.919614,gry
1,112370,2016-04-03 19:33:34,6087575183,"['Odzież, Obuwie, Dodatki', 'Odzież i bielizna...",1,1,TheBestLook,0,1,0,0,0,68.514680,"odzież, obuwie, dodatki"
2,277719,2016-04-03 17:23:26,5880072408,"['Telefony i Akcesoria', 'Akcesoria GSM', 'Fol...",1,0,pawelec01,2,1,1,0,1,90.270269,telefony i akcesoria
3,243899,2016-04-03 09:56:18,5252897056,"['Dom i Ogród', 'Ogród', 'Narzędzia ogrodnicze...",1,1,cencik-tools,0,1,2,0,0,122.458265,dom i ogród
4,111123,2016-04-03 16:13:14,5804280383,"['Telefony i Akcesoria', 'Akcesoria GSM', 'Pok...",1,1,KoshibaZ-T,0,1,0,0,0,91.314765,telefony i akcesoria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205804,113631,2016-04-03 19:10:06,5935546262,"['Motoryzacja', 'Części samochodowe', 'Oświetl...",1,0,AgnaBoas,0,1,0,0,0,45.137251,motoryzacja
205805,268456,2016-04-03 14:32:14,6096748337,"['Uroda', 'Kosmetyki pielęgnacyjne', 'Ciało', ...",1,0,ECO-FUD,0,1,2,0,0,44.722719,uroda
205806,416308,2016-04-03 15:19:32,5271125404,"['Odzież, Obuwie, Dodatki', 'Odzież damska', '...",1,0,studio_anet,0,1,1,0,1,72.079900,"odzież, obuwie, dodatki"
205807,253825,2016-04-03 11:48:41,6089547624,"['Motoryzacja', 'Części do maszyn', 'Części do...",1,0,Farmexpert,0,0,2,0,0,15.247662,motoryzacja


### Encoding zmiennej *main_category*
Do zakodowania *main_cateogry* użyłyśmy metody One Hot Encoding, bo unikatowych wartości tej kolumny nie jest tak wiele.

In [170]:
encoder = ce.OneHotEncoder(cols="main_category", use_cat_names=True)
allegro_train_tr = encoder.fit_transform(allegro_train_tr)
allegro_train_tr.head()

,lp,date,item_id,categories,pay_option_on_delivery,pay_option_transfer,seller,price,it_is_allegro_standard,it_quantity,...,main_category_instrumenty,main_category_muzyka,main_category_biżuteria i zegarki,main_category_antyki i sztuka,"main_category_sprzęt estradowy, studyjny i dj-ski",main_category_fotografia,main_category_konsole i automaty,main_category_nieruchomości,main_category_bilety,main_category_filmy
0,245512,2016-04-03 16:04:48,6091183856,"['Gry', 'Gry na konsole', 'Pegasus']",1,1,Sajgon6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,112370,2016-04-03 19:33:34,6087575183,"['Odzież, Obuwie, Dodatki', 'Odzież i bielizna...",1,1,TheBestLook,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,277719,2016-04-03 17:23:26,5880072408,"['Telefony i Akcesoria', 'Akcesoria GSM', 'Fol...",1,0,pawelec01,2,1,1,...,0,0,0,0,0,0,0,0,0,0
3,243899,2016-04-03 09:56:18,5252897056,"['Dom i Ogród', 'Ogród', 'Narzędzia ogrodnicze...",1,1,cencik-tools,0,1,2,...,0,0,0,0,0,0,0,0,0,0
4,111123,2016-04-03 16:13:14,5804280383,"['Telefony i Akcesoria', 'Akcesoria GSM', 'Pok...",1,1,KoshibaZ-T,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Wstępna walidacja z próbki budującej
Sprawdźmy czy przyjęte sposoby uzupełniania braków danych, kategoryzacji i encodingu są uniwersalne dla otrzymanych danych.

In [171]:
allegro_train_tst.describe()

,lp,item_id,pay_option_on_delivery,pay_option_transfer,price,it_is_allegro_standard,it_quantity,it_is_brand_zone,it_seller_rating
count,88205.000000,8.820500e+04,88205.000000,88205.000000,88205.000000,88205.000000,88205.000000,88205.000000,88205.000000
mean,209910.861935,5.810374e+09,0.875710,0.812108,76.988211,0.583674,6601.741557,0.016779,20280.966056
std,121219.203934,3.388177e+08,0.329914,0.390628,375.137928,0.492952,23097.049826,0.128444,36378.010670
min,2.000000,4.179879e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000
25%,104628.000000,5.654928e+09,1.000000,1.000000,9.000000,0.000000,6.000000,0.000000,1583.000000
50%,209634.000000,5.968788e+09,1.000000,1.000000,24.990000,1.000000,94.000000,0.000000,6690.000000
75%,315098.000000,6.071330e+09,1.000000,1.000000,68.000000,1.000000,931.000000,0.000000,20813.000000
max,420015.000000,6.099913e+09,1.000000,1.000000,64900.000000,1.000000,99999.000000,1.000000,292074.000000


###  Uzupełnienie braków
Wartości podejrzane o oznaczanie braku danych są takie same jak w próbce treningowej

In [172]:
impute_missing(allegro_train_tst)

### Encoding zmiennej *it-location*

In [173]:
allegro_train_tst['it_location']=allegro_train_tst['it_location'].str.lower()
allegro_train_tst['main_category']=allegro_train_tst['main_category'].str.lower()

In [174]:
encoder=TargetEncoder()
allegro_train_tst['it_location']=encoder.fit_transform(allegro_train_tst['it_location'], allegro_train_tst['price'])
allegro_train_tst

,lp,date,item_id,categories,pay_option_on_delivery,pay_option_transfer,seller,price,it_is_allegro_standard,it_quantity,it_is_brand_zone,it_seller_rating,it_location,main_category
0,116128,2016-04-03 20:55:47,6042531265,"['Dla Dzieci', 'Odzież', 'Kurtki, płaszcze', '...",0,1,KAT-MODA,50.00,0,232,0,408,69.019500,dla dzieci
1,289135,2016-04-03 14:12:18,6097286903,"['RTV i AGD', 'AGD drobne', 'Higiena i pielęgn...",1,0,robziel,30.00,0,232,0,57,65.801370,rtv i agd
2,322646,2016-04-03 19:35:27,5074717838,"['Dom i Ogród', 'Utrzymanie czystości', 'Susza...",1,1,lampka_alladyna,29.00,1,983,0,13057,63.226077,dom i ogród
3,313677,2016-04-03 22:39:07,6069685251,"['Odzież, Obuwie, Dodatki', 'Odzież damska', '...",1,1,orek24,294.99,0,1,0,1960,81.951078,"odzież, obuwie, dodatki"
4,146424,2016-04-03 20:11:08,6048233675,"['Dom i Ogród', 'Ogród', 'Meble ogrodowe', 'Pa...",1,1,ZiAndrzej,249.00,0,980,0,60798,79.395460,dom i ogród
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88200,396978,2016-04-03 23:32:45,6036877026,"['Dom i Ogród', 'Ogród', 'Rośliny', 'Rośliny o...",1,1,kampas2,7.00,0,26,0,5954,126.996396,dom i ogród
88201,318134,2016-04-03 10:16:46,6092853417,"['Gry', 'Gry online (MMO)', 'Pozostałe']",0,1,Pixelobsessions,33.49,0,8527,0,1292,44.067922,gry
88202,291587,2016-04-03 18:12:38,5463639713,"['Uroda', 'Kosmetyki pielęgnacyjne', 'Pielęgna...",1,1,kosmetyki_ck,11.95,0,98930,0,154017,54.001500,uroda
88203,364688,2016-04-03 19:46:13,6066935746,"['Odzież, Obuwie, Dodatki', 'Obuwie', 'Akcesor...",1,1,bacolihit,43.00,0,25,0,2963,107.334603,"odzież, obuwie, dodatki"


### Kategoryzacja i encoding zmiennych ciągłych

In [175]:
allegro_train_tst.groupby(by='main_category').describe().loc[:,["price","it_quantity"]]

price                                 \
                                       count        mean          std   min   
main_category                                                                 
antyki i sztuka                        238.0   96.750756   261.646442  1.00   
bilety                                  22.0   39.876364    96.837410  1.00   
biuro i reklama                        671.0   36.676051    96.709166  1.00   
biżuteria i zegarki                   1230.0   53.786846   183.722984  1.00   
delikatesy                            1775.0   18.173313    27.400109  1.00   
dla dzieci                            8876.0   72.568578   138.330020  1.00   
dom i ogród                          19106.0   71.441198   177.367966  1.00   
filmy                                  222.0   30.398919    29.379999  1.00   
fotografia                             520.0  137.908519   411.489862  1.00   
gry                                   1565.0   41.053412    60.314091  1.00   
instrumenty                            141.0  116.813050   205.728233  1.00   
kolekcje                              1279.0   45.879414   158.384144  1.00   
komputery                             3058.0  112.201089   316.041260  1.00   
konsole i automaty                     214.0  156.024019   341.630204  1.00   
książki i komiksy                     2366.0   24.468859    27.152077  1.00   
motoryzacja                           9554.0  138.024898  1014.706392  1.00   
muzyka                                 415.0   36.303422    53.357266  1.00   
odzież, obuwie, dodatki              11430.0   75.617799    82.088641  1.00   
przemysł                              1198.0   83.808097   307.913973  1.00   
rtv i agd                             4195.0  116.634570   316.648599  1.00   
rękodzieło                            1390.0    7.902986    16.970532  1.00   
sport i turystyka                     5735.0   94.642806   214.672546  1.00   
sprzęt estradowy, studyjny i dj-ski     90.0   71.541889   101.580594  1.25   
telefony i akcesoria                  4220.0   83.361059   255.966543  1.00   
uroda                                 5913.0   27.986031    41.941873  1.00   
zdrowie                               2782.0   42.225582    77.428197  1.00   

                                                                          \
                                         25%     50%       75%       max   
main_category                                                              
antyki i sztuka                      13.6250  29.990   77.2425   3000.00   
bilety                                3.6250  17.250   20.7500    459.00   
biuro i reklama                       4.0000  11.300   27.9000   1450.00   
biżuteria i zegarki                   5.9900  14.800   42.9900   5320.00   
delikatesy                            4.9500  10.000   20.9000    389.98   
dla dzieci                           12.9900  29.990   79.9900   3198.00   
dom i ogród                           6.0000  18.000   65.0000   5284.00   
filmy                                 9.9100  19.990   37.9000    149.00   
fotografia                           14.5000  29.000   67.0000   4899.00   
gry                                   7.0000  23.490   54.9900   1340.00   
instrumenty                          15.9000  39.000  108.0000   1514.00   
kolekcje                              4.9000  12.990   35.4250   3275.00   
komputery                            12.9900  33.165   79.9900   3999.00   
konsole i automaty                   11.9925  39.495  102.6750   2349.00   
książki i komiksy                     9.0000  18.830   28.9000    366.66   
motoryzacja                          15.0000  38.900   99.0000  64900.00   
muzyka                               15.0000  25.000   39.9900    909.00   
odzież, obuwie, dodatki              19.9900  41.990  109.0000   1600.00   
przemysł                              3.8925  19.000   62.8000   6700.00   
rtv i agd                             6.7350  27.900   92.9950   6790.00   

In [176]:
categorize_price(allegro_train_tst)
categorize_quantity(allegro_train_tst)
categorize_rating(allegro_train_tst)

In [177]:
# Sprawdzenie ilości unikatowych wartośći w każdej kolumnie
allegro_train_tst.nunique()

lp                        88205
date                      27628
item_id                   81178
categories                 6775
pay_option_on_delivery        2
pay_option_transfer           2
seller                    24856
price                         3
it_is_allegro_standard        2
it_quantity                   3
it_is_brand_zone              2
it_seller_rating              2
it_location                2732
main_category                26
dtype: int64

In [178]:
encoding(allegro_train_tst)
allegro_train_tst

,lp,date,item_id,categories,pay_option_on_delivery,pay_option_transfer,seller,price,it_is_allegro_standard,it_quantity,it_is_brand_zone,it_seller_rating,it_location,main_category
0,116128,2016-04-03 20:55:47,6042531265,"['Dla Dzieci', 'Odzież', 'Kurtki, płaszcze', '...",0,1,KAT-MODA,0,0,0,0,0,69.019500,dla dzieci
1,289135,2016-04-03 14:12:18,6097286903,"['RTV i AGD', 'AGD drobne', 'Higiena i pielęgn...",1,0,robziel,0,0,0,0,0,65.801370,rtv i agd
2,322646,2016-04-03 19:35:27,5074717838,"['Dom i Ogród', 'Utrzymanie czystości', 'Susza...",1,1,lampka_alladyna,0,1,1,0,1,63.226077,dom i ogród
3,313677,2016-04-03 22:39:07,6069685251,"['Odzież, Obuwie, Dodatki', 'Odzież damska', '...",1,1,orek24,1,0,2,0,0,81.951078,"odzież, obuwie, dodatki"
4,146424,2016-04-03 20:11:08,6048233675,"['Dom i Ogród', 'Ogród', 'Meble ogrodowe', 'Pa...",1,1,ZiAndrzej,1,0,1,0,1,79.395460,dom i ogród
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88200,396978,2016-04-03 23:32:45,6036877026,"['Dom i Ogród', 'Ogród', 'Rośliny', 'Rośliny o...",1,1,kampas2,0,0,2,0,1,126.996396,dom i ogród
88201,318134,2016-04-03 10:16:46,6092853417,"['Gry', 'Gry online (MMO)', 'Pozostałe']",0,1,Pixelobsessions,0,0,1,0,0,44.067922,gry
88202,291587,2016-04-03 18:12:38,5463639713,"['Uroda', 'Kosmetyki pielęgnacyjne', 'Pielęgna...",1,1,kosmetyki_ck,0,0,1,0,1,54.001500,uroda
88203,364688,2016-04-03 19:46:13,6066935746,"['Odzież, Obuwie, Dodatki', 'Obuwie', 'Akcesor...",1,1,bacolihit,0,0,0,0,0,107.334603,"odzież, obuwie, dodatki"


Kategoryzacja i encoding powiodły się. Określenie dobrego i złego sprzedawcy z przybliżeniem do 5000 dla mediany (tym razem 6690) ponownie uznajemy za rozsądne.

### Encoding zmiennej *main_category*

In [179]:
encoder = ce.OneHotEncoder(cols="main_category", use_cat_names=True)
allegro_train_tst = encoder.fit_transform(allegro_train_tst)
allegro_train_tst.head()

,lp,date,item_id,categories,pay_option_on_delivery,pay_option_transfer,seller,price,it_is_allegro_standard,it_quantity,...,main_category_przemysł,main_category_muzyka,main_category_biżuteria i zegarki,main_category_fotografia,main_category_instrumenty,main_category_konsole i automaty,main_category_filmy,"main_category_sprzęt estradowy, studyjny i dj-ski",main_category_antyki i sztuka,main_category_bilety
0,116128,2016-04-03 20:55:47,6042531265,"['Dla Dzieci', 'Odzież', 'Kurtki, płaszcze', '...",0,1,KAT-MODA,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,289135,2016-04-03 14:12:18,6097286903,"['RTV i AGD', 'AGD drobne', 'Higiena i pielęgn...",1,0,robziel,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,322646,2016-04-03 19:35:27,5074717838,"['Dom i Ogród', 'Utrzymanie czystości', 'Susza...",1,1,lampka_alladyna,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,313677,2016-04-03 22:39:07,6069685251,"['Odzież, Obuwie, Dodatki', 'Odzież damska', '...",1,1,orek24,1,0,2,...,0,0,0,0,0,0,0,0,0,0
4,146424,2016-04-03 20:11:08,6048233675,"['Dom i Ogród', 'Ogród', 'Meble ogrodowe', 'Pa...",1,1,ZiAndrzej,1,0,1,...,0,0,0,0,0,0,0,0,0,0


Wszystkie encodingi przebiegły pomyślnie.

# Wnioski

Podsumowując naszą klasyfikację zbioru *allegro-api-transactions* warto zauważyć kilka kluczowych kwestii:
1. Podział na próbkę treningową i walidacyjną przeprowadziłyśmy w trochę inny sposób niż zazwyczaj. Jako że zmienna celu *price* jest zmienną ciągłą, dopiero później ją kategoryzujemy, miała ona zbyt dużo unikalnych rekordów, by móc klasycznie rozdzielić nasze dane. Stąd użyłyśmy funkcji *scsplit* z pakietu przeznaczonego do podziału próbki przy wyżej opisanej sytuacji.
2. Nasz zbiór z pozoru nie miał braków danych, ale podejrzane wartości (0 i -1) wystąpiły dla 3 kolumn: *price*, *it_quantity*, *it_seller_rating*. We wszytkich trzech przypadkach zastąpiłyśmy je medianą z pozostałych rekordów dla każdej zmiennej.
3. Przeprowadziłyśmy kategoryzację zmiennych ciągłych: price, it_quantity oraz it_seller_rating dwóm pierwszym przyporządkowując kategorie *low*, *average* i *high*, a ostatniej *bad* oraz *good*. Kategorie *low*, *average* i *high* przyporządkowałyśmy biorąc pod uwagę podział na kwantyle (low - <25%, average - 25%-75%, high - >75%), a przy kategoryzacji zmiennej *it_seller_rating* szacowałyśmy granicę podziału w okolicah mediany.
4. Jeśli chodzi o encoding, zrobiłyśmy go klasycznie dla trzech wymienionych wyżej zmiennych, a także dla zmiennej *location* i *main_category*. Przy zmiennej *main_category* dobrym wyjściem okazał się one-hot encoding, jako że mamy stosunkowo małą ilość (27) unikalnych rekordów, zaś przy *location* zastosowałyśmy target encoding z uwagi na to, że tutaj mamy znacznie więcej różniących się wyników, a nie dodaje nam on zbędnych kolumn, a więc nie zwiększa rozmiaru ramki danych. Nie jest to rozwiązanie pozbawione wad, ale o ile zmienna *price* w zbiorze walidacyjnym nie będzie miała znacznie innego rozkładu, nie powinno dojść do przeuczenia.